In [ ]:
import os
import sys
sys.coinit_flags = 2
import glob
import pathlib
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14 # グラフの基本フォントサイズの設定
import tkinter as tk
import tkinter.filedialog as fd

class App(tk.Frame):
    def __init__(self, master = None):
        tk.Frame.__init__(self, master)
        self.pack()
        self.create_widgets()
        
    def create_widgets(self):
        ### 1.ファイルを選択する
        # ラベル
        self.label1 = tk.Label(self, text = '1. csvファイルを選択して下さい\n（下記にファイルパスが表示されます）',
                               font=("Arial", 10, "bold"))
        self.label1.grid(row = 0, column = 0, sticky=tk.W)
        # ファイル選択ボタン（引数「command」の内容を処理する）
        self.btn1 = tk.Button(self, text = 'ファイル選択', command = self.btn1_open_file)
        self.btn1.grid(row = 1, column = 1)
        # エントリー（パスの表示する場所。1行の場合はEntryメソッド、複数行の場合はTextを使う）
        self.entry1 = tk.Entry(self, width = 70)
        self.entry1.grid(row = 1, column = 0, padx=10)
        
        
        ### 2.フォルダを選択する
        # ラベル
        self.label2 = tk.Label(self, text = '2. フォルダを選択して下さい\n（下記にサブフォルダ一覧が表示されます）',
                               font=("Arial", 10, "bold"))
        self.label2.grid(row = 2, column = 0, sticky=tk.W)
        # フォルダ選択ボタン（引数「command」の内容を処理する）
        self.btn2 = tk.Button(self, text = 'フォルダ選択', command = self.btn2_select_folder)
        self.btn2.grid(row = 3, column = 1)
        # テキスト（読み込んだデータを表示する場所。複数行の場合はテキスト、1行の場合はEntryメソッドを使う）
        self.text2 = tk.Text(self, width=60, height=10)
        self.text2.grid(row = 4, column = 0, columnspan = 2, sticky = tk.W + tk.E, padx = 10)
        
        
        ### 3.ラジオボタンによる選択
        # ラベル
        self.label3 = tk.Label(self, text = '3. 次のいずれかのボタンを選択して下さい\n（ラジオボタンの設置は3つ以上でも可能です）',
                               font=("Arial", 10, "bold"),
                               anchor = 'e', justify = 'left')
        self.label3.grid(row = 6, column = 0, sticky = tk.W)
        
        # ラジオボタンの値を取得してプリント表示する関数
        def button_clicked_func():
            print(self.val.get())
        
        # ラジオボタンの値の識別を整数型で設定
        self.val = tk.IntVar()
        self.val.set(0) # デフォルト値は「0」で設定

        # ラジオボタン用のラベル
        self.radio_label1 = tk.Label(self, text = 'Onの場合', font=("Arial", 10, "bold"), bg = 'cyan')
        self.radio_label1.grid(row = 7, column = 0)
        self.radio_label2 = tk.Label(self, text = 'Offの場合', font=("Arial", 10, "bold"), bg = 'orange')
        self.radio_label2.grid(row = 7, column = 1)        
        
        # ラジオボタンの設定
        self.radio_btn1 = tk.Radiobutton(self, variable = self.val, value = 0, command = button_clicked_func)
        self.radio_btn1.grid(row = 8, column = 0)
        self.radio_btn2 = tk.Radiobutton(self, variable = self.val, value = 1, command = button_clicked_func)
        self.radio_btn2.grid(row = 8, column = 1) 
        
        
        ### 4.ユーザーのキーボード入力を入力ボックスで引数として受け付ける
        # ラベル
        self.label4 = tk.Label(self, text = '4. キーボード入力を下記で受け付けます', font=("Arial", 10, "bold"))
        self.label4.grid(row = 9, column = 0, sticky = tk.W)
        # エントリー（入力を受け付ける場所）
        self.entry4 = tk.Entry(self, width = 70)
        self.entry4.grid(row = 10, column = 0, columnspan = 2, sticky = tk.W + tk.E, padx = 10)
        
        
        ### 5.ボタンを押すと、自作関数を実行するコマンド
        self.label5 = tk.Label(self, text = '5. 1～4で取得した値をテキストファイルへ書き出します', font=("Arial", 10, "bold"))
        self.label5.grid(row = 11, column = 0, sticky = tk.W)
        self.btn5 = tk.Button(self, text = '実行ボタン', command = self.run_cmd)
        self.btn5.grid(row = 12, column = 0)
        

    # ファイル選択メソッド
    # ここでは、選択したファイルのパスを「self.file_path1」へ代入する。そして、エントリーエリアへ表示する.
    # もし、選択と同時にしたい処理があれば、そのコードを追記する
    def btn1_open_file(self):
        self.entry1.delete(0, tk.END)
        #filetype1 = [('カンマ', '*.csv'), ('エクセル', '*.xlsx'), ('すべて', '*')]
        filetype1 = [('カンマ', '*.csv'), ('すべて', '*')]
        
        # ファイルパスを取得
        self.file_path1 = fd.askopenfilename(initialdir = os.path.expanduser('~/Desktop'), filetypes = filetype1)
        
        # 選択したファイルパスをエントリーエリアに表示
        self.entry1.insert(0, self.file_path1)

    # フォルダ選択メソッド
    # ここでは、選択したフォルダ以下のサブフォルダのパス一覧を「self.get_folder_list」リストへ格納する.そして、エントリーエリアへ表示する.
    # もし、選択と同時にしたい処理があれば、そのコードを追記する
    def btn2_select_folder(self):
        # すでに入力されてる場合、一旦削除
        self.text2.delete('1.0', tk.END)
        
        # フォルダパスの取得
        self.parent_folder_path = fd.askdirectory(initialdir = os.path.expanduser('~/Desktop'))      
        
        # 再帰的に、サブフォルダのパスを取得する場合
        get_folder_list = list(pathlib.Path(self.parent_folder_path).glob('**'))
#         # 再帰的に、サブフォルダとファイルも取得する場合
#         get_folder_and_file_list = list(pathlib.Path(self.parent_folder_path).glob('**/*'))
        
        # 親フォルダ以外のパスをリストで抽出する
        sub_folder_list = get_folder_list[1:]
        # Pathオブジェクトを文字列へ変換する
        self.sub_folder_list = [str(p) for p in sub_folder_list]  
            
        # フォルダパスのリストに対して、要素間に改行コードを付与して、文字列へ変換
        sub_folder_str = '\n'.join(self.sub_folder_list)
        
        # 取得したフォルダパスの一覧をテキストエリアに表示
        self.text2.insert('end', sub_folder_str)
        
    # 実行メソッド
    def run_cmd(self):
        save_file_path = os.getcwd() + r'/output_data'
        
        ### 1.ファイル選択メソッドで、取得したファイルパスよりファイルの中を読み出し、特定の情報を書き出す
        # pandasデータフレームで読み出す
        df = pd.read_csv(self.file_path1)

        # 基本統計量を出力
        df_describe = df.describe().T
        df_describe.to_csv(f'{save_file_path}_1_describe.csv', index = False)
        
        # ヒストグラムの表示
        fig = plt.figure(figsize=(10,8))
        ax = fig.gca()
        df.hist(ax=ax)
        plt.tight_layout()
        #plt.show()
        plt.savefig(f'{save_file_path}_1_hisotgram.jpg',)
        plt.close()
        
        ### 2.フォルダ選択メソッドで取得したフォルダ内にあるサブディレクトリパスのリストを書き出す
        with open(f'{save_file_path}_2.txt', 'w') as f:
            f.writelines(f'{self.sub_folder_list}\n') 
        
        ### 3.ラジオボタンで選択されてる値に応じて、if文で分岐した値を書き出す
        with open(f'{save_file_path}_3.txt', 'w') as f:
            if self.val.get() == 0:
                _val = '「Onの場合」が選択されました'
            else:
                _val = '「Offの場合」が選択されました'
            f.write(_val + '\n') 
            
        ### 4.入力ボックスを文字列へ変換(数値の場合もありうるため)
        with open(f'{save_file_path}_4.txt', 'w') as f:        
            _mystr = str(self.entry4.get())
            f.write(f'{_mystr}\n')

        ### 実行の終了を表示する
        self.label_end = tk.Label(self, text = '実行ボタンの処理を完了しました',
                                  font=("Helvetica", 12, "bold"), fg = 'blue')
        self.label_end.grid(row = 13, column = 0, columnspan = 2, sticky = tk.W + tk.E)            
            
if __name__ == '__main__':
    root = tk.Tk()
    root.title('TKinterで作成したアプリケーション')
    root.geometry('640x480') # アプリのサイズ
    #root.configure(bg='red')
    
    frame = App(root)
    frame.pack(pady=5, fill=tk.X)    
    frame.mainloop()
    